In [1]:
# source https://github.com/tfjgeorge/ift6268/blob/master/projet/FF%20texture%20generation%20bricks%20-%20Adam%20blocks%20training.ipynb

In [1]:
import theano
import numpy

from theano import tensor
from fuel.streams import ServerDataStream

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from models.ann_2_blocks import build_model
from blocks.algorithms import GradientDescent, Adam
from blocks.graph import ComputationGraph

Using gpu device 0: GRID K520 (CNMeM is disabled, CuDNN 3007)


LOAD NEURAL NETWORK AND LOSS

In [2]:
images = tensor.ftensor4('images')
labels = tensor.lmatrix('labels')
cost   = build_model(images.dimshuffle(0,3,1,2), labels)

LEARN WEIGHTS

In [3]:
train_stream = ServerDataStream(('images', 'labels'), False, hwm=10)
valid_stream = ServerDataStream(('images', 'labels'), False, hwm=10, port=5558)

In [4]:
cg = ComputationGraph(cost)

algorithm = GradientDescent(
	cost=cost,
	parameters=cg.parameters,
	step_rule=Adam()
)

In [5]:
from blocks.extensions import Printing, Timing
from blocks.extensions.training import TrackTheBest
from blocks.extensions.monitoring import TrainingDataMonitoring, DataStreamMonitoring
from blocks.extensions.stopping import FinishIfNoImprovementAfter
from blocks.extensions.saveload import Checkpoint
from blocks_extras.extensions.plot import Plot
import socket
import datetime
import time

host_plot = 'http://localhost:5006'
cost.name = 'cost'

extensions = [
    Timing(),
    TrainingDataMonitoring([cost], after_epoch=True),
    DataStreamMonitoring(variables=[cost], data_stream=valid_stream, prefix="valid"),
    Plot('ann_2 %s' % (datetime.datetime.now(), ),channels=[['cost'],['valid_cost']], after_epoch=True, server_url=host_plot),
    TrackTheBest('valid_cost'),
    Checkpoint('train'),
    FinishIfNoImprovementAfter('loss_best_so_far', epochs=5),
    Printing()
]

Using saved session configuration for http://localhost:5006
To override, pass 'load_from_config=False' to Session


In [6]:
from blocks.main_loop import MainLoop


main_loop = MainLoop(data_stream=train_stream, algorithm=algorithm,
                     extensions=extensions)

In [ ]:
main_loop.run()


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 time_initialization: 71.1842660904
	 valid_cost: 53.7663879395

